In [23]:
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from IPython.display import clear_output
import pandas as pd

### Starting Drivers

In [139]:
#Open ChromeDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get("https://www.google.com/")



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\harik\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
<ipython-input-139-ff91a2323aef>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [77]:
#Open Firefox Driver
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
# driver.get("https://www.google.com/")



====== WebDriver manager ======
Current firefox version is 94.0
Get LATEST geckodriver version for 94.0 firefox
Getting latest mozilla release info for v0.30.0
Trying to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.30.0/geckodriver-v0.30.0-win64.zip
Driver has been saved in cache [C:\Users\harik\.wdm\drivers\geckodriver\win64\v0.30.0]
<ipython-input-77-03aa60d10d31>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


### Sign Up for 1lib.in
1. Go to https://singlelogin.org/registration.php <br>
2. Get mail, password and nickname input and fill it<br>
3. Click on the "Sign Up" button <br>
4. Save the email and password


In [135]:
### emails
login_details = pd.read_csv('login_details.csv')
emails = list(login_details['email'])
passwords = list(login_details['password'])

In [133]:
def register(email):
    """
    Registers a new user on https://singlelogin.org/registration.php
    returns the email address of the user
    """
    #Step 1
    driver.get("https://singlelogin.org/registration.php")

    #Step 2
    #Email
    email_entry = driver.find_element(By.NAME, 'email')
    email_entry.clear()
    email_entry.send_keys(email)

    #Password
    password = email[:8]
    password_entry = driver.find_element(By.NAME, 'password')
    password_entry.clear()
    password_entry.send_keys(password)

    #Step 3
    #Submit Button
    submit_button = driver.find_element(By.TAG_NAME, 'form button')
    submit_button.click()

    try:
        emails.append(email)
        passwords.append(password)
        df = pd.DataFrame(data=[emails, passwords]).T
        df.columns = ['email', 'password']
        df.to_csv('login_details.csv', index=False)
    except:
        print("You should add this email to database")
        return email
    return email

In [136]:
register('vakofit832@elastit.com')

'vakofit832@elastit.com'

### Login to 1lib.in
1. Go to https://singlelogin.org/login.php <br>
2. Get mail and password input and fill it <br>
3. Click on the "Login" button <br>

In [ ]:
def login(email):
    """
    Logs in to https://singlelogin.org/
    """
    #Step 1
    driver.get("https://singlelogin.org/?from=1lib.in")

    #Step 2
    #Email
    email_entry = driver.find_element(By.NAME, 'email')
    email_entry.clear()
    email_entry.send_keys(email)

    #Password
    password = email[:8]
    password_entry = driver.find_element(By.NAME, 'password')
    password_entry.clear()
    password_entry.send_keys(password)

    #Step 3
    #Submit Button
    submit_button = driver.find_element(By.TAG_NAME, 'form button')
    submit_button.click()

### Steps to Download Book from 1lib.in
1. Go to Homepage <br>
2. Search for the book <br>
3. Select one book (usually first hit) <br>
4. Download the book <br>


In [47]:
# book_name = "The Book Thief"
def book_download(book_name):
    """
    Downloads the book if 5 book quota is not reached using https://1lib.in/
    returns the download link of the book
    """
    #Get to homepage
    clear_output(wait=True)
    print(f"Trying to download {book_name}")
    print("Going to Zlibrary")
    driver.get('https://1lib.in/')
    print("Done!")

    print("Searching for the book")
    #Search for book
    driver.find_element(By.ID, 'searchFieldx').clear()
    driver.find_element(By.ID, "searchFieldx").send_keys(book_name.strip().title())
    driver.find_element(By.ID, "searchFieldx").send_keys(Keys.ENTER)
    print("Done!")

    #Click on the First hit if it exist
    try:
        book_link = driver.find_element(By.CLASS_NAME, 'exactMatch').find_element(By.TAG_NAME, 'a').get_attribute('href')
        driver.get(book_link)
        print("Found the book!")
    except:
        print("No exact match")
        return None
    #Click on download button
    download_link = driver.find_element(By.CLASS_NAME, 'btn-group').find_element(By.TAG_NAME, 'a').get_attribute('href')
    
    driver.get(download_link)
    if 'WARNING: There are more than 5' in driver.find_element(By.CLASS_NAME, 'container').text[:50]:
        print("5 books are already downloaded so the book can not be downloaded.")
        print("Please wait for 24 hours.")
        print("The download link is returned.")
        return download_link
    else:
        print("Book should be downloading!")
        return download_link

In [48]:
book_download("Pride and Prejudice")

Trying to download Pride and Prejudice
Going to Zlibrary
Done!
Searching for the book
Done!
Found the book!
Book should be downloading!


'https://1lib.in/dl/1642163/5cfdd3'

In [137]:
driver.close()

### Temp-Mail
Since libgen gives just 5 books per day, we need to use a temporary email address to download more books.

#### Steps 1
1. Go to https://temp-mail.org/en/ <br>
2. Get the Temporary Email Address <br>

#### Steps 2
Verify the email address by clicking on the link in the email. <br>

In [115]:
def get_temp_email():
    #Step 1
    driver.get("https://temp-mail.org/en/")

    #Copying to clipboard
    driver.find_element(By.CLASS_NAME, "copyIconGreenBtn").click()

    #Getting email using pandas
    email = pd.read_clipboard().columns[0]
    return email